# Hierarchical Multi-Agent System

This system implements a hierarchical organizational structure:
- **CEO** (top level supervisor)
  - **Research Team Leader** (manages researchers)
    - Data Researcher
    - Market Researcher
  - **Writing Team Leader** (manages writers)
    - Technical Writer
    - Summary Writer


In [13]:
# Imports
from typing import TypedDict, Annotated, Literal, Dict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END, MessagesState
from langchain_openai import ChatOpenAI
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize LLM
llm = ChatOpenAI(model="gpt-4.1", temperature=0)


In [14]:
# ===================================
# State Definition
# ===================================

class HierarchicalState(MessagesState):
    """State for the hierarchical multi-agent system"""
    next_agent: str = ""
    current_task: str = ""
    
    # Research team outputs
    data_research: str = ""
    market_research: str = ""
    research_complete: bool = False
    
    # Writing team outputs
    technical_report: str = ""
    summary_report: str = ""
    writing_complete: bool = False
    
    # Final output
    final_output: str = ""
    task_complete: bool = False


In [3]:
# ===================================
# CEO Agent (Top Level Supervisor)
# ===================================

def ceo_agent(state: HierarchicalState) -> Dict:
    """CEO coordinates the entire organization"""
    
    messages = state["messages"]
    task = messages[-1].content if messages else "No task"
    
    # Check what's been completed
    has_research = state.get("research_complete", False)
    has_writing = state.get("writing_complete", False)
    
    # CEO decision logic
    if has_writing:
        # Everything is done, compile final output
        technical_report = state.get("technical_report", "")
        summary_report = state.get("summary_report", "")
        
        final_output = f"""
╔══════════════════════════════════════════════════════════════╗
║                    EXECUTIVE SUMMARY REPORT                  ║
╚══════════════════════════════════════════════════════════════╝

Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Task: {task}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📋 EXECUTIVE SUMMARY
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{summary_report}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 TECHNICAL DETAILS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

{technical_report}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Report compiled by Hierarchical Multi-Agent AI System
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""
        
        return {
            "messages": [AIMessage(content="👔 CEO: All teams have completed their work. Final report compiled!")],
            "final_output": final_output,
            "task_complete": True,
            "next_agent": "end"
        }
    elif not has_research:
        # Research not done, assign to Research Team Leader
        return {
            "messages": [AIMessage(content="👔 CEO: Assigning task to Research Team Leader.")],
            "current_task": task,
            "next_agent": "research_team_leader"
        }
    elif not has_writing:
        # Research done, assign to Writing Team Leader
        return {
            "messages": [AIMessage(content="👔 CEO: Research complete. Assigning to Writing Team Leader.")],
            "next_agent": "writing_team_leader"
        }
    else:
        return {
            "messages": [AIMessage(content="👔 CEO: Task complete.")],
            "next_agent": "end"
        }


In [4]:
# ===================================
# Research Team Leader
# ===================================

def research_team_leader(state: HierarchicalState) -> Dict:
    """Research Team Leader coordinates Data Researcher and Market Researcher"""
    
    data_research = state.get("data_research", "")
    market_research = state.get("market_research", "")
    task = state.get("current_task", "")
    
    # Check what research is done
    has_data_research = bool(data_research)
    has_market_research = bool(market_research)
    
    if not has_data_research:
        # Assign to Data Researcher
        return {
            "messages": [AIMessage(content="🔬 Research Team Leader: Assigning Data Researcher to gather technical data.")],
            "next_agent": "data_researcher"
        }
    elif not has_market_research:
        # Assign to Market Researcher
        return {
            "messages": [AIMessage(content="🔬 Research Team Leader: Assigning Market Researcher to gather market insights.")],
            "next_agent": "market_researcher"
        }
    else:
        # Both researchers done, mark research complete
        return {
            "messages": [AIMessage(content="🔬 Research Team Leader: All research complete. Reporting to CEO.")],
            "research_complete": True,
            "next_agent": "ceo"
        }


In [5]:
# ===================================
# Data Researcher
# ===================================

def data_researcher(state: HierarchicalState) -> Dict:
    """Data Researcher gathers technical data and statistics"""
    
    task = state.get("current_task", "research topic")
    
    # Create data research prompt
    data_prompt = f"""As a Data Researcher, provide comprehensive technical data and statistics about: {task}

Focus on:
1. Quantitative data and metrics
2. Technical specifications and details
3. Statistical analysis
4. Data-driven insights
5. Numbers, percentages, and measurable facts

Be precise and data-focused."""
    
    # Get research from LLM
    research_response = llm.invoke([HumanMessage(content=data_prompt)])
    data_research = research_response.content
    
    # Create agent message
    agent_message = f"📊 Data Researcher: Completed data research on '{task}'.\n\nKey data points:\n{data_research[:300]}..."
    
    return {
        "messages": [AIMessage(content=agent_message)],
        "data_research": data_research,
        "next_agent": "research_team_leader"
    }


In [6]:
# ===================================
# Market Researcher
# ===================================

def market_researcher(state: HierarchicalState) -> Dict:
    """Market Researcher gathers market insights and trends"""
    
    task = state.get("current_task", "research topic")
    
    # Create market research prompt
    market_prompt = f"""As a Market Researcher, provide comprehensive market insights about: {task}

Focus on:
1. Market trends and dynamics
2. Competitive landscape
3. Consumer behavior and preferences
4. Market opportunities and challenges
5. Industry analysis and positioning

Be strategic and market-focused."""
    
    # Get research from LLM
    research_response = llm.invoke([HumanMessage(content=market_prompt)])
    market_research = research_response.content
    
    # Create agent message
    agent_message = f"📈 Market Researcher: Completed market research on '{task}'.\n\nKey market insights:\n{market_research[:300]}..."
    
    return {
        "messages": [AIMessage(content=agent_message)],
        "market_research": market_research,
        "next_agent": "research_team_leader"
    }


In [7]:
# ===================================
# Writing Team Leader
# ===================================

def writing_team_leader(state: HierarchicalState) -> Dict:
    """Writing Team Leader coordinates Technical Writer and Summary Writer"""
    
    technical_report = state.get("technical_report", "")
    summary_report = state.get("summary_report", "")
    
    # Check what writing is done
    has_technical = bool(technical_report)
    has_summary = bool(summary_report)
    
    if not has_technical:
        # Assign to Technical Writer
        return {
            "messages": [AIMessage(content="✍️ Writing Team Leader: Assigning Technical Writer to create detailed report.")],
            "next_agent": "technical_writer"
        }
    elif not has_summary:
        # Assign to Summary Writer
        return {
            "messages": [AIMessage(content="✍️ Writing Team Leader: Assigning Summary Writer to create executive summary.")],
            "next_agent": "summary_writer"
        }
    else:
        # Both writers done, mark writing complete
        return {
            "messages": [AIMessage(content="✍️ Writing Team Leader: All writing complete. Reporting to CEO.")],
            "writing_complete": True,
            "next_agent": "ceo"
        }


In [8]:
# ===================================
# Technical Writer
# ===================================

def technical_writer(state: HierarchicalState) -> Dict:
    """Technical Writer creates detailed technical report"""
    
    data_research = state.get("data_research", "")
    market_research = state.get("market_research", "")
    task = state.get("current_task", "")
    
    # Create technical writing prompt
    technical_prompt = f"""As a Technical Writer, create a comprehensive technical report based on:

Task: {task}

Data Research:
{data_research[:1500]}

Market Research:
{market_research[:1500]}

Create a detailed technical report with:
1. Technical Overview
2. Data Analysis
3. Technical Specifications
4. Implementation Details
5. Technical Recommendations

Be thorough, technical, and detailed."""
    
    # Get report from LLM
    report_response = llm.invoke([HumanMessage(content=technical_prompt)])
    technical_report = report_response.content
    
    # Create agent message
    agent_message = f"📝 Technical Writer: Completed technical report.\n\nReport preview:\n{technical_report[:300]}..."
    
    return {
        "messages": [AIMessage(content=agent_message)],
        "technical_report": technical_report,
        "next_agent": "writing_team_leader"
    }


In [9]:
# ===================================
# Summary Writer
# ===================================

def summary_writer(state: HierarchicalState) -> Dict:
    """Summary Writer creates executive summary"""
    
    data_research = state.get("data_research", "")
    market_research = state.get("market_research", "")
    technical_report = state.get("technical_report", "")
    task = state.get("current_task", "")
    
    # Create summary writing prompt
    summary_prompt = f"""As a Summary Writer, create an executive summary based on:

Task: {task}

Data Research:
{data_research[:800]}

Market Research:
{market_research[:800]}

Technical Report:
{technical_report[:800]}

Create a concise executive summary with:
1. Key Highlights
2. Main Findings
3. Critical Insights
4. Strategic Recommendations
5. Conclusion

Be concise, clear, and executive-focused."""
    
    # Get report from LLM
    report_response = llm.invoke([HumanMessage(content=summary_prompt)])
    summary_report = report_response.content
    
    # Create agent message
    agent_message = f"📄 Summary Writer: Completed executive summary.\n\nSummary preview:\n{summary_report[:300]}..."
    
    return {
        "messages": [AIMessage(content=agent_message)],
        "summary_report": summary_report,
        "next_agent": "writing_team_leader"
    }


In [10]:
# ===================================
# Router Function
# ===================================

def router(state: HierarchicalState) -> Literal["ceo", "research_team_leader", "data_researcher", "market_researcher", "writing_team_leader", "technical_writer", "summary_writer", "__end__"]:
    """Routes to next agent based on state"""
    
    next_agent = state.get("next_agent", "ceo")
    
    if next_agent == "end" or state.get("task_complete", False):
        return "__end__"
        
    if next_agent in ["ceo", "research_team_leader", "data_researcher", "market_researcher", "writing_team_leader", "technical_writer", "summary_writer"]:
        return next_agent
        
    return "ceo"


In [11]:
# ===================================
# Build the Hierarchical Graph
# ===================================

# Create workflow
workflow = StateGraph(HierarchicalState)

# Add nodes (in hierarchical order)
workflow.add_node("ceo", ceo_agent)
workflow.add_node("research_team_leader", research_team_leader)
workflow.add_node("data_researcher", data_researcher)
workflow.add_node("market_researcher", market_researcher)
workflow.add_node("writing_team_leader", writing_team_leader)
workflow.add_node("technical_writer", technical_writer)
workflow.add_node("summary_writer", summary_writer)

# Set entry point
workflow.set_entry_point("ceo")

# Add conditional routing for all nodes
for node in ["ceo", "research_team_leader", "data_researcher", "market_researcher", "writing_team_leader", "technical_writer", "summary_writer"]:
    workflow.add_conditional_edges(
        node,
        router,
        {
            "ceo": "ceo",
            "research_team_leader": "research_team_leader",
            "data_researcher": "data_researcher",
            "market_researcher": "market_researcher",
            "writing_team_leader": "writing_team_leader",
            "technical_writer": "technical_writer",
            "summary_writer": "summary_writer",
            "__end__": END
        }
    )

# Compile the graph
hierarchical_graph = workflow.compile()

print("✅ Hierarchical Multi-Agent Graph Created Successfully!")


✅ Hierarchical Multi-Agent Graph Created Successfully!


In [12]:
# ===================================
# Test the Hierarchical System
# ===================================

response = hierarchical_graph.invoke({
    "messages": [HumanMessage(content="Analyze the impact of AI on modern business operations")]
})

print("\n" + "="*70)
print("WORKFLOW COMPLETED")
print("="*70 + "\n")

# Display final output
if 'final_output' in response and response['final_output']:
    print(response['final_output'])
else:
    print("Final output not yet generated.")
    print("\nCurrent state keys:", list(response.keys()))
    print("\nTask complete:", response.get('task_complete', False))
    print("\nLast message:", response.get('messages', [])[-1].content if response.get('messages') else 'No messages')


KeyboardInterrupt: 

In [ ]:
# View all messages from the workflow
print("\n" + "="*70)
print("WORKFLOW MESSAGES")
print("="*70 + "\n")

for i, msg in enumerate(response.get('messages', []), 1):
    print(f"{i}. {msg.content}\n")
